### 기업 고유코드

In [106]:
import requests

url = 'https://opendart.fss.or.kr/api/corpCode.xml'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
params = {
    'crtfc_key': api_key,
}

response = requests.get(url, params=params)
if response.status_code != 200:     # URL GET '200 정상'
    print("URL GET Error")


In [107]:
import io
import zipfile

# 출력포멧이 Zip FILE (binary)
f = io.BytesIO(response.content)
zfile = zipfile.ZipFile(f)

In [108]:
zfile.namelist()

['CORPCODE.xml']

In [109]:
import xmltodict
# 압축된 파일에서 데이터를 읽습니다. xmltodict 모듈을 사용해서 압축이 해제된 xml 데이터를 파이썬의 OrderedDict 타입으로 변환합니다.
xml_data = zfile.read("CORPCODE.xml").decode("utf-8")
dict_data = xmltodict.parse(xml_data)

In [110]:
dict_data

{'result': {'list': [{'corp_code': '00434003',
    'corp_name': '다코',
    'stock_code': None,
    'modify_date': '20170630'},
   {'corp_code': '00434456',
    'corp_name': '일산약품',
    'stock_code': None,
    'modify_date': '20170630'},
   {'corp_code': '00430964',
    'corp_name': '굿앤엘에스',
    'stock_code': None,
    'modify_date': '20170630'},
   {'corp_code': '00432403',
    'corp_name': '한라판지',
    'stock_code': None,
    'modify_date': '20170630'},
   {'corp_code': '00388953',
    'corp_name': '크레디피아제이십오차유동화전문회사',
    'stock_code': None,
    'modify_date': '20170630'},
   {'corp_code': '00179984',
    'corp_name': '연방건설산업',
    'stock_code': None,
    'modify_date': '20170630'},
   {'corp_code': '00420143',
    'corp_name': '브룩스피알아이오토메이션잉크',
    'stock_code': None,
    'modify_date': '20170630'},
   {'corp_code': '00401111',
    'corp_name': '매경아이비아이',
    'stock_code': None,
    'modify_date': '20170630'},
   {'corp_code': '00435534',
    'corp_name': '캐드뱅크',
    'stock_code': Non

In [111]:
import pandas as pd
# 딕셔너리에서 키 값을 확인하여 실제 데이터가 있는 부분을 인덱싱합니다. 그러다음 해당 데이터를 판다스의 데이터프레임 객체로 변환합니다.
data_list = dict_data['result']['list']
df_corp = pd.DataFrame(data_list)

In [112]:
df_corp.head()

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,None,20170630
1,00434456,일산약품,None,20170630
2,00430964,굿앤엘에스,None,20170630
3,00432403,한라판지,None,20170630
4,00388953,크레디피아제이십오차유동화전문회사,None,20170630


In [113]:
corp_name = ['대성에너지', '삼천리', '경동도시가스', '서울도시가스', '인천도시가스', '부산도시가스', '씨엔씨티에너지']
df_corp.query('corp_name in @corp_name').sort_values('corp_name')

,corp_code,corp_name,stock_code,modify_date
81145,01208885,경동도시가스,267290,20230215
78606,00787376,대성에너지,117580,20230213
63144,00123967,부산도시가스,015350,20220217
85341,00128555,삼천리,004690,20230106
76600,00130684,서울도시가스,017390,20221205
82706,00152011,씨엔씨티에너지,None,20221214
98000,00145862,인천도시가스,034590,20230320


In [125]:
df_corp.query('corp_name in @corp_name').sort_values('corp_name')['corp_code'].to_list()

['01208885',
 '00787376',
 '00123967',
 '00128555',
 '00130684',
 '00152011',
 '00145862']

In [114]:
# 회사명에 특정 문자열을 포함한 목록 찾기
df_corp[df_corp['corp_name'].str.contains('씨엔씨티')]

,corp_code,corp_name,stock_code,modify_date
82706,00152011,씨엔씨티에너지,None,20221214


In [115]:
# import requests
# from bs4 import BeautifulSoup
# # import xml.etree.ElementTree as ET
# import json
# import pandas as pd

# params = {
#     'crtfc_key': api_key,
# }

# response = requests.get(url, params=params)
# if response.status_code == 200:
#     soup = BeautifulSoup(response.text, "html.parser")
#     # jo = json.loads(response.text)
# else: 
#     print(response.status_code)
# # result = pd.json_normalize(jo['StatisticSearch']['row'])
# # result
# # try:
# #     tree = ET.XML(r.content)
# #     status = tree.find('status').text
# #     message = tree.find('message').text
# #     if status != '000':
# #         raise ValueError({'status': status, 'message': message})
# # except ET.ParseError as e:
# #     print("Error")
# #     # jo = r.json()
# #     # if jo['status'] != '000':
# #     #     print(ValueError(r.text))

In [116]:
# print(soup.prettify())

In [117]:
# tags = soup.find_all("td")